In [123]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sb, joblib

In [191]:
# 데이터 불러오기
df = pd.read_csv("../data/고객db_전처리.csv")

In [192]:
# 차량 구매 서비스 플랫폼 -> 회원 정보, 구매 정보 포함
df

,연번,이름,생년월일,현재 나이,연령대,성별,휴대폰 번호,이메일,고객 구분,아이디,...,최근 구매 시점,최근 구매 요일,마지막 구매 이후 경과 기간,최근 구매 당시 나이,최근 거래 금액,선호 거래 방식,선호 구매 경로,차량 구매 횟수,신차 구매 지수,브랜드 충성도 지수
0,1,강주온,2002-03-04,23,20대 중반,남,010-6502-8753,janisi@himedia.co.kr,법인,14798,...,2024 2분기,평일,350,22,40410000,신용카드,오프라인,1,0.002,0.135
1,2,김경수,1983-01-02,42,40대 초반,남,010-1111-2222,kimkyoungsoo31@himedia.co.kr,법인,kyoungsoo031,...,2025 1분기,평일,38,42,89730000,신용카드,오프라인,5,0.508,0.037
2,3,김다은,1999-05-02,26,20대 중반,여,010-6789-0123,kimdaeun26@himedia.co.kr,법인,daeun026,...,2025 1분기,평일,10,26,70920000,계좌이체,오프라인,1,0.161,0.004
3,4,김민수,1960-07-01,65,60대 중반,남,010-1234-5678,kimminsu1@himedia.co.kr,법인,minsu001,...,2023 2분기,평일,689,63,36110000,현금,오프라인,1,0.002,0.130
4,5,김승현,1970-03-02,55,50대 중반,남,010-0123-4567,kimseunghyun20@himedia.co.kr,법인,seunghyun020,...,2023 4분기,주말,536,53,45460000,신용카드,오프라인,1,0.004,0.103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,288,신시우,1958-07-11,67,60대 후반,여,042-207-6955,gyeonghyi78@gmail.com,개인,jihungim,...,2024 4분기,평일,90,66,85410000,신용카드,온라인,2,0.011,0.280
287,289,최춘자,1976-06-27,49,40대 후반,여,031-170-7739,hancunja@yu.kr,개인,jinho50,...,2023 3분기,평일,574,47,37750000,현금,오프라인,3,0.047,0.116
288,290,차서준,1968-01-22,57,50대 후반,여,064-741-3740,qgim@simryujo.com,개인,jihun41,...,2024 1분기,평일,433,56,71500000,계좌이체,온라인,2,0.005,0.304
289,291,안정희,1984-01-10,41,40대 초반,남,051-406-1803,donghyeonbae@joyunson.com,개인,yeongil47,...,2024 1분기,평일,415,40,84630000,현금,온라인,2,0.006,0.268


# 예측 모델 개발

In [193]:
# 전처리 기본 세팅
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [194]:
df

,연번,이름,생년월일,현재 나이,연령대,성별,휴대폰 번호,이메일,고객 구분,아이디,...,최근 구매 시점,최근 구매 요일,마지막 구매 이후 경과 기간,최근 구매 당시 나이,최근 거래 금액,선호 거래 방식,선호 구매 경로,차량 구매 횟수,신차 구매 지수,브랜드 충성도 지수
0,1,강주온,2002-03-04,23,20대 중반,남,010-6502-8753,janisi@himedia.co.kr,법인,14798,...,2024 2분기,평일,350,22,40410000,신용카드,오프라인,1,0.002,0.135
1,2,김경수,1983-01-02,42,40대 초반,남,010-1111-2222,kimkyoungsoo31@himedia.co.kr,법인,kyoungsoo031,...,2025 1분기,평일,38,42,89730000,신용카드,오프라인,5,0.508,0.037
2,3,김다은,1999-05-02,26,20대 중반,여,010-6789-0123,kimdaeun26@himedia.co.kr,법인,daeun026,...,2025 1분기,평일,10,26,70920000,계좌이체,오프라인,1,0.161,0.004
3,4,김민수,1960-07-01,65,60대 중반,남,010-1234-5678,kimminsu1@himedia.co.kr,법인,minsu001,...,2023 2분기,평일,689,63,36110000,현금,오프라인,1,0.002,0.130
4,5,김승현,1970-03-02,55,50대 중반,남,010-0123-4567,kimseunghyun20@himedia.co.kr,법인,seunghyun020,...,2023 4분기,주말,536,53,45460000,신용카드,오프라인,1,0.004,0.103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,288,신시우,1958-07-11,67,60대 후반,여,042-207-6955,gyeonghyi78@gmail.com,개인,jihungim,...,2024 4분기,평일,90,66,85410000,신용카드,온라인,2,0.011,0.280
287,289,최춘자,1976-06-27,49,40대 후반,여,031-170-7739,hancunja@yu.kr,개인,jinho50,...,2023 3분기,평일,574,47,37750000,현금,오프라인,3,0.047,0.116
288,290,차서준,1968-01-22,57,50대 후반,여,064-741-3740,qgim@simryujo.com,개인,jihun41,...,2024 1분기,평일,433,56,71500000,계좌이체,온라인,2,0.005,0.304
289,291,안정희,1984-01-10,41,40대 초반,남,051-406-1803,donghyeonbae@joyunson.com,개인,yeongil47,...,2024 1분기,평일,415,40,84630000,현금,온라인,2,0.006,0.268


In [195]:
# X, y 분할
X = df[["최근 거래 금액", "거주 지역", "차량 사이즈", "차량 유형", "연료 구분"]]
y = df.loc[df["연료 구분"] == "휘발유", "최근 구매 제품"]

In [196]:
# X = 최근 거래 금액, 거주 지역, 차량 사이즈, 차량 유형, 연료 구분
# y = 최근 구매 제품

In [197]:
X

,최근 거래 금액,거주 지역,차량 사이즈,차량 유형,연료 구분
0,40410000,인천광역시,준중형,세단,휘발유
1,89730000,광주광역시,대형,세단,휘발유
2,70920000,부산광역시,중형,SUV,수소
3,36110000,전라남도 목포시,중형,SUV,디젤
4,45460000,경기도 수원시,중형,세단,휘발유
...,...,...,...,...,...
286,85410000,전라북도 전주시,대형,세단,휘발유
287,37750000,충청남도 천안시,준중형,세단,휘발유
288,71500000,경상남도 창원시,중형,SUV,수소
289,84630000,대구광역시,대형,세단,휘발유


In [198]:
y

0      Avante (CN7 N)
1           G80 (RG3)
4            G70 (IK)
5            G70 (IK)
9            G90 (HI)
            ...      
284         G80 (RG3)
286         G80 (RG3)
287    Avante (CN7 N)
289         G80 (RG3)
290         G80 (RG3)
Name: 최근 구매 제품, Length: 162, dtype: object

In [199]:
# 인코더 생성
label = LabelEncoder()
onehot = OneHotEncoder()

In [200]:
X = pd.get_dummies(X, columns=["거주 지역", "차량 사이즈", "차량 유형", "연료 구분"])

In [201]:
X

,최근 거래 금액,거주 지역_경기도 성남시,거주 지역_경기도 수원시,거주 지역_경상남도 창원시,거주 지역_경상북도 포항시,거주 지역_광주광역시,거주 지역_대구광역시,거주 지역_대전광역시,거주 지역_부산광역시,거주 지역_서울특별시,...,차량 사이즈_프리미엄,차량 유형_SUV,차량 유형_세단,차량 유형_해치백,연료 구분_디젤,연료 구분_수소,연료 구분_전기,연료 구분_플러그인 하이브리드,연료 구분_하이브리드,연료 구분_휘발유
0,40410000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1,89730000,False,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,70920000,False,False,False,False,False,False,False,True,False,...,False,True,False,False,False,True,False,False,False,False
3,36110000,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False
4,45460000,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,85410000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
287,37750000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
288,71500000,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
289,84630000,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [202]:
X = X.loc[X["연료 구분_휘발유"] == True, :]

In [203]:
X

,최근 거래 금액,거주 지역_경기도 성남시,거주 지역_경기도 수원시,거주 지역_경상남도 창원시,거주 지역_경상북도 포항시,거주 지역_광주광역시,거주 지역_대구광역시,거주 지역_대전광역시,거주 지역_부산광역시,거주 지역_서울특별시,...,차량 사이즈_프리미엄,차량 유형_SUV,차량 유형_세단,차량 유형_해치백,연료 구분_디젤,연료 구분_수소,연료 구분_전기,연료 구분_플러그인 하이브리드,연료 구분_하이브리드,연료 구분_휘발유
0,40410000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1,89730000,False,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
4,45460000,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
5,46320000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
9,136370000,False,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,87550000,False,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
286,85410000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
287,37750000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
289,84630000,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [204]:
X = X.iloc[:, :24]

In [205]:
X

,최근 거래 금액,거주 지역_경기도 성남시,거주 지역_경기도 수원시,거주 지역_경상남도 창원시,거주 지역_경상북도 포항시,거주 지역_광주광역시,거주 지역_대구광역시,거주 지역_대전광역시,거주 지역_부산광역시,거주 지역_서울특별시,...,거주 지역_충청남도 천안시,거주 지역_충청북도 청주시,차량 사이즈_대형,차량 사이즈_준대형,차량 사이즈_준중형,차량 사이즈_중형,차량 사이즈_프리미엄,차량 유형_SUV,차량 유형_세단,차량 유형_해치백
0,40410000,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
1,89730000,False,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
4,45460000,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
5,46320000,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
9,136370000,False,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,87550000,False,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
286,85410000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
287,37750000,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,True,False
289,84630000,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [206]:
y

0      Avante (CN7 N)
1           G80 (RG3)
4            G70 (IK)
5            G70 (IK)
9            G90 (HI)
            ...      
284         G80 (RG3)
286         G80 (RG3)
287    Avante (CN7 N)
289         G80 (RG3)
290         G80 (RG3)
Name: 최근 구매 제품, Length: 162, dtype: object

# 데이터셋 분할

In [207]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [208]:
# 고객 추천 모델 --> Classification
# GBM, Linear SVM, CatBoost, KNN, GaussianNB, MultinomialNB, BernoulliNB, MLP 필요
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [209]:
lr = LogisticRegression()
svc = SVC()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()
gnb = GaussianNB()
knn = KNeighborsClassifier()
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
lgb = LGBMClassifier()
cb = CatBoostClassifier()

In [210]:
lr.fit(train_X, train_y)
svc.fit(train_X, train_y)
dtc.fit(train_X, train_y)
rfc.fit(train_X, train_y)
gbc.fit(train_X, train_y)
gnb.fit(train_X, train_y)
knn.fit(train_X, train_y)
lgb.fit(train_X, train_y)
cb.fit(train_X, train_y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000187 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50
[LightGBM] [Info] Number of data points in the train set: 129, number of used features: 4
[LightGBM] [Info] Start training from score -1.170933
[LightGBM] [Info] Start training from score -1.031171
[LightGBM] [Info] Start training from score -1.248894
[LightGBM] [Info] Start training from score -4.859812
[LightGBM] [Info] Start training from score -4.859812
[LightGBM] [Info] Start training from score -3.473518
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

In [211]:
y_pred_lr = lr.predict(test_X)
y_pred_svc = svc.predict(test_X)
y_pred_dtc = dtc.predict(test_X)
y_pred_rfc = rfc.predict(test_X)
y_pred_gbc = gbc.predict(test_X)
y_pred_gnb = gnb.predict(test_X)
y_pred_knn = knn.predict(test_X)
y_pred_lgb = lgb.predict(test_X)
y_pred_cb = cb.predict(test_X)

In [212]:
# 정확도 확인
print("Logistic Regression 정확도 : ", accuracy_score(test_y, y_pred_lr))
print("SVM 정확도 : ", accuracy_score(test_y, y_pred_svc))
print("Decision Tree 정확도 : ", accuracy_score(test_y, y_pred_dtc))
print("Random Forest 정확도 : ", accuracy_score(test_y, y_pred_rfc))
print("Gradient Boosting 정확도 : ", accuracy_score(test_y, y_pred_gbc))
print("Gaussian Naive Bayes 정확도 : ", accuracy_score(test_y, y_pred_gnb))
print("KNN 정확도 : ", accuracy_score(test_y, y_pred_knn))
print("LightGBM 정확도 : ", accuracy_score(test_y, y_pred_lgb))
print("CatBoost 정확도 : ", accuracy_score(test_y, y_pred_cb))

Logistic Regression 정확도 :  0.2727272727272727
SVM 정확도 :  1.0
Decision Tree 정확도 :  1.0
Random Forest 정확도 :  1.0
Gradient Boosting 정확도 :  1.0
Gaussian Naive Bayes 정확도 :  1.0
KNN 정확도 :  1.0
LightGBM 정확도 :  1.0
CatBoost 정확도 :  1.0


In [213]:
joblib.dump(dtc, "models/DecisionTree 휘발유 모델.pkl")
joblib.dump(rfc, "models/RandomForest 휘발유 모델.pkl")
joblib.dump(gbc, "models/GradientBoosting 휘발유 모델.pkl")
joblib.dump(lgb, "models/LightGBM 휘발유 모델.pkl")

['models/LightGBM 휘발유 모델.pkl']

### dtc, rfc, gbc, lgb, cb 모델의 정확도가 높음

In [91]:
test_X

,최근 거래 금액,거주 지역_경기도 성남시,거주 지역_경기도 수원시,거주 지역_경상남도 창원시,거주 지역_경상북도 포항시,거주 지역_광주광역시,거주 지역_대구광역시,거주 지역_대전광역시,거주 지역_부산광역시,거주 지역_서울특별시,거주 지역_울산광역시,거주 지역_인천광역시,거주 지역_전라남도 목포시,거주 지역_전라북도 전주시,거주 지역_충청북도 청주시,차량 사이즈_준대형,차량 사이즈_중형,차량 유형_SUV
3,36110000,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True
32,48180000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True
152,45760000,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True
206,41220000,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True
55,38290000,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True
272,51030000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True
132,39850000,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True
224,48990000,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True
52,52310000,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True
281,42800000,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True


In [46]:
new_data = np.array([35000000, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 0,0,0,1,0, 1,0,0]).reshape(1, 24)

In [47]:
recom_list = []

In [48]:
recom_list.append(dtc.predict(new_data)[0])
recom_list.append(rfc.predict(new_data)[0])
recom_list.append(gbc.predict(new_data)[0])
recom_list.append(lgb.predict(new_data)[0])

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [49]:
recom_list

['Santa-Fe ™', 'Santa-Fe ™', 'Santa-Fe ™', 'Santa-Fe ™', 'Santa-Fe ™']

In [50]:
recom_list = list(set(recom_list))

In [51]:
# 추천 결과 하나씩 보여주기
for i in recom_list :
    print(i)

Santa-Fe ™


# 비슷한 가격대 전기차 3대 추천

In [52]:
elec_car_compen = {
    "서울특별시": 9000000,
    "부산광역시": 10500000,
    "대구광역시": 11000000,
    "인천광역시": 10600000,
    "광주광역시": 11000000,
    "대전광역시": 12000000,
    "울산광역시": 10500000,
    "경기도 수원시": 10500000,
    "경기도 성남시": 11000000,
    "충청북도 청주시": 14000000,
    "충청남도 천안시": 14000000,
    "전라북도 전주시": 15000000,
    "전라남도 목포시": 15500000,
    "경상북도 포항시": 13000000,
    "경상남도 창원시": 13000000
}

if "연료 구분" in ["전기", "플러그인 하이브리드", "하이브리드"]:
     pass
else: # 추천 시작
     compen = elec_car_compen["서울특별시"]
     # df.loc[df["최근 거래 금액"] <= 45000000 + compen, "최근 구매 제품"].to_numpy()[0].head(3)

In [53]:
compen = elec_car_compen["서울특별시"]
recom_elec = df.loc[(df["최근 거래 금액"] <= 45000000 + compen) & (df["연료 구분"].isin(["전기", "플러그인 하이브리드", "하이브리드"])), "최근 구매 제품"].to_numpy()[0:3]

In [54]:
for i in recom_elec:
    print(i)

IONIQ 6 (CE)
Grandeur (GN7 HEV)
Tucson (NX4 PHEV)


# 프로모션 : 신용카드 혜택 설명하면서 50% 비싼 차 추천 -> 옵션 붙인 걸로 생각하면 됨

In [55]:
prom_data = np.array([35000000 * 1.5, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 0,0,0,1,0, 1,0,0]).reshape(1, 24)

In [56]:
prom_list = []

In [57]:
prom_list.append(dtc.predict(prom_data)[0])
prom_list.append(rfc.predict(prom_data)[0])
prom_list.append(gbc.predict(prom_data)[0])
prom_list.append(lgb.predict(prom_data)[0])

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [58]:
prom_list = list(set(prom_list))

In [59]:
prom_list

['Santa-Fe ™']

In [60]:
# 프로모션 결과와 추천 결과 비교해서, 같은 결과는 제외하고 보여주기
# 모두 다 같으면 애초에 프로모션 안 함
for i in prom_list :
    if i not in recom_list :
        print(i)